Okay! Let's try the gym business. Cliff walking environment is perfect.

In [1]:
import gym
import numpy as np
import ToyEnv
import psrl
import PosteriorSamplerGMM

In [2]:
env = gym.make('CliffWalking-v0')

In [3]:
observation, info = env.reset()
print(observation)
print(info)
np.unravel_index(observation, env.shape)

36
{'prob': 1}


(3, 0)

In [4]:
action = env.action_space.sample()
print(action)
observation, reward, terminated, truncated, info = env.step(3)
print(observation)
print(env.action_space.n)
print(reward)
print(env.P[observation][2])

2
36
4
-1
[(1.0, 36, -1, False)]


/home/srayan/miniconda3/envs/tsmdp/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# Testing ToyEnv

In [2]:
new_env = ToyEnv.ToyEnv(3, 3, 2)
init_state = new_env.reset()
print(init_state)

0


In [3]:
next_state, reward = new_env.step(1)
print(reward)
print(next_state)

6.013403329662627
2


# Testing Gibbs sampler

In [4]:
# Generate some data for state-action pair (0, 2)
means = new_env._compute_means(0, 2)
sigma = new_env.true_Sigma
pis = new_env.true_pi
data = np.zeros((100,))
for i in range(100):
    k = np.random.choice(len(pis), p=pis)
    data[i] = np.random.normal(loc=means[k], scale=sigma[k])

In [17]:
print(means)
print(sigma)
print(pis)

[2 5]
[0.1 0.1]
[0.2 0.8]


In [22]:
gibbs = PosteriorSamplerGMM.GibbsSamplerGMM(1000, 2)

In [23]:
samples = gibbs.fit(data)

In [25]:
print(samples[-500])

(array([5.00015311, 1.9834373 ]), array([0.06634279, 0.17205711]), array([0.82502349, 0.17497651]))


# Testing PSRL

In [4]:
alg = psrl.PSRL(new_env)


In [5]:
_, _, policy = alg.run()

Iteration: 0
[]
Iteration: 1
s, a (0, 0)
relevant_hist: []
No data provided. Sampling from prior.
Sample 0
Sample 1
Sample 2
Sample 3
Sample 4
Sample 5
Sample 6
Sample 7
Sample 8
Sample 9
Sample 10
Sample 11
Sample 12
Sample 13
Sample 14
Sample 15
Sample 16
Sample 17
Sample 18
Sample 19
Sample 20
Sample 21
Sample 22
Sample 23
Sample 24
Sample 25
Sample 26
Sample 27
Sample 28
Sample 29
Sample 30
Sample 31
Sample 32
Sample 33
Sample 34
Sample 35
Sample 36
Sample 37
Sample 38
Sample 39
Sample 40
Sample 41
Sample 42
Sample 43
Sample 44
Sample 45
Sample 46
Sample 47
Sample 48
Sample 49
Sample 50
Sample 51
Sample 52
Sample 53
Sample 54
Sample 55
Sample 56
Sample 57
Sample 58
Sample 59
Sample 60
Sample 61
Sample 62
Sample 63
Sample 64
Sample 65
Sample 66
Sample 67
Sample 68
Sample 69
Sample 70
Sample 71
Sample 72
Sample 73
Sample 74
Sample 75
Sample 76
Sample 77
Sample 78
Sample 79
Sample 80
Sample 81
Sample 82
Sample 83
Sample 84
Sample 85
Sample 86
Sample 87
Sample 88
Sample 89
Sample 90
Sa

ValueError: setting an array element with a sequence.